# Redis Product Catalog

## Product Catalog Use Case

In the last session several common Redis use cases were discussed. Now, lets consider using Redis as our primary database for a product catalog for a new online store. It will need to store product details including a name, description, vendor, price, main category and some images.

## Requirements:

* Product information stored in the database should include: name, description, vendor, price, category, images associated with that product.
* Ability to create/update/delete product details
* Ability to find product by ID
* Ability to find products in category X
* Ability to find product by its name or part of its name

## Step 3 - Logical Data Model:

The logical data model is separate from the DBMS being used.
It defines the structure of data elements and to set relationships between them.

* Product Image
    * Id: Number
    * Value: Binary

* Product
    * Id: Number
    * Name: String
    * Description: String
    * Vendor: String
    * Price: Number
    * Currency: String
    * MainCategory: Category (1)
    * Images: Image (0..n)
 
* Category
    * id: Number
    * Name: String
    * Products: Product (0..n)

In [1]:
# Lets make some sample products in a json:

In [1]:
# Product Image
product_image = {
"Id": 1,
"Value": 'binaryvalue'
}

In [2]:
# Product
product = {
    'Id': 1,
    'Name': 'ProductA',
    'Description': 'Chicken',
    'Vendor': 'Walmart',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 1, # mapped to category (Id)
    'Images': [1,2,3,4] # mapped to product_image (Id)
}

In [3]:
# Category
category = {
    "Id": 1,
    "Name": 'ProductA',
    "Products": [1,2,3] # mapped to product (Id)
}

## Step 4 - Convert to Physical Data Model for Redis:

How would you take this logical data model and turn it into a physical model in Redis?

What data structures would you use to meet the requirements listed above?

How to index and query the data?

Be ready to show via redis-cli (or Insight) how you stored the data and how you are able to find the product by ID, list all prodcuts in a given category and find a Product by full or partial name.

In [165]:
# import redis
import redis
import os

# connect redis to localhost and port
redis = redis.Redis(host='localhost', port=6379, db=0)

In [166]:
# product image: what should we store it as in redis?
product_image

{'product_image:1457441629': {'Value': 100}}

In [167]:
# it looks like we can store it as a simple String (key value pair)
# we will need to fix the structure how how we deal with this though.

In [168]:
# lets make a product image data structure in python
def product_image_kv(product_id, value):
    product_image = {f"product_image:{product_id}":{
    'Value': value,
    }}
    return product_image

In [169]:
# lets create some values
import random

random.seed()
product_id = random.getrandbits(32)
value = 100

product_image = product_image_kv(product_id, value)
product_image

{'product_image:1471133404': {'Value': 100}}

In [170]:
# lets convert it to a redis data structure (a string, a key value)
for Id, product_dict in product_image.items():
    #print(Id, product_dict)
    for k,v in product_dict.items():
            #redis.set(Id,v)
            #print(k,v)
            redis.set(Id,v)
    #check it out
    print(Id)
    print(redis.get(Id))
    

product_image:1471133404
b'100'


In [171]:
# lets make the Category data model

In [172]:
print(category)

Meat


In [173]:
# we will need to follow a similar process that we did with the product image.
# this time, we cant use a string data structure, we need a different one.
# a Hash is really the best option

# the challenge here is that we need to map in all the product ids.

In [174]:
def category_hash(category_id, name, products):
    category = {f"category:{category_id}":{
    'Name': name,
    'Products': products
    }}
    return category

In [175]:
import random

random.seed()
category_id = random.getrandbits(32)
name = 'Meat'
products = [1,2,5,6,7]

category = category_hash(category_id, name, products)
category

{'category:1254762292': {'Name': 'Meat', 'Products': [1, 2, 5, 6, 7]}}

In [176]:
# lets convert it to a redis data structure (a string, a key value)
for Id, category_dict in category.items():
    #print(Id, category_dict)
    for k,v in category_dict.items():
            #print(k,v)
            if k != 'Products':
                redis.hset(Id,k,v)
            else:
                #print(v)
                # add products to a new data structure with a set and a new id
                for i in v:
                    #print(i)
                    set_id_products = f"{Id}:products"
                    redisList = redis.sadd(set_id_products,i)
            #redis.hset(Id,k,v)
    #check it out
    print(Id)
    print(redis.hgetall(Id))
    print(set_id_products)
    print(redis.smembers(set_id_products))

category:1254762292
{b'Name': b'Meat'}
category:1254762292:products
{b'7', b'2', b'5', b'6', b'1'}


In [177]:
redis.flushdb()

True

In [179]:
#redis.scan()

In [180]:
# lets make a product data model

# we will use a hash function.

# the challenge here is the we need the image and category to fit into this model.

In [181]:
def product_hash(product_id, name, description, vendor, price, currency, mainCategory, images):
    product = {f"product:{product_id}":{
    'Name': name,
    'Description': description,
    'Vendor': vendor,
    'Price': price,
    'Currency': currency,
    'MainCategory': mainCategory, # mapped to category (Id)
    'Images':  images# mapped to product_image (Id)
    }}
    return product

In [182]:
import random

random.seed()
product_id = random.getrandbits(32)
name = 'ProductA'
description = 'Chicken'
vendor = 'CostCo'
price = 12
currency = 'dollars'
mainCategory = [1,5,67,8]
images = [1,2,3,4]

products = product_hash(product_id, name, description, vendor, price, currency, mainCategory, images)

In [183]:
# lets convert it to a redis data structure (a string, a key value)
for Id, product_dict in products.items():
    #print(Id, product_dict)
    for k,v in product_dict.items():
            #print(k,v)
            if k != 'MainCategory' and k != 'Images':
                redis.hset(Id,k,v)
            elif k == 'MainCategory':
                #print(v)
                # add products to a new data structure with a set and a new id
                for i in v:
                    #print(i)
                    set_id_maincat = f"{Id}:maincat"
                    redis.sadd(set_id_maincat,i)
            elif k == 'Images':
                #print(v)
                # add products to a new data structure with a set and a new id
                for i in v:
                    #print(i)
                    set_id_images = f"{Id}:images"
                    redis.sadd(set_id_images,i)
            #redis.hset(Id,k,v)
    #check it out
    print(Id)
    print(redis.hgetall(Id))
    print(set_id_maincat)
    print(redis.smembers(set_id_maincat))
    print(set_id_images)
    print(redis.smembers(set_id_images))

product:3748912688
{b'Name': b'ProductA', b'Description': b'Chicken', b'Vendor': b'CostCo', b'Price': b'12', b'Currency': b'dollars'}
product:3748912688:maincat
{b'5', b'67', b'1', b'8'}
product:3748912688:images
{b'2', b'4', b'1', b'3'}


In [195]:
redis.scan(0,'product*')

(0, [b'product:3748912688:images'])

In [257]:
redis.incr('cat', 1)
redis.get('cat')

3

In [259]:
redis.get('cat')

b'3'

In [201]:
name = 'cat'
if redis.sadd('product-names',name) == 1:
    print('yes')


In [198]:
redis.smembers('product-names')

{b'cat'}

In [192]:
redis.hget('product:3748912688','Name')

b'ProductA'

In [557]:
redis.zadd('my-images',{10:3})

1

In [565]:
redis.zadd('my-images',{'value1':0})
redis.zrevrange('my-images',0,4,withscores=True)

[(b'binaryvalue4', 1.0), (b'value1', 0.0)]

In [568]:
redis.zincrby('my-images',1,'value1')

2.0

In [605]:
image_zset_id = 'my-images'
image_binary_val = 'myBinary5'
print(redis.zcard(image_zset_id))
if redis.zcard(image_zset_id) > 0:
    top_score=redis.zrevrange(image_zset_id,0,0,withscores=True)
    top_score_incr = top_score[0][1] + 1
    redis.zadd(image_zset_id,{image_binary_val:top_score_incr})
else:
    redis.zadd(image_zset_id,{image_binary_val:0})

6


In [606]:
redis.zrevrange(image_zset_id,0,-1,withscores=True)

[(b'myBinary5', 6.0),
 (b'myBinary4', 4.0),
 (b'myBinary3', 3.0),
 (b'myBinary2', 2.0),
 (b'myBinary1', 1.0),
 (b'myBinary', 0.0)]

In [634]:
image_list_id = 'my-images2'
image_binary_val = 'myBinary6'

redis.lpush(image_list_id,image_binary_val)
redis.ltrim(image_list_id,0,3)

redis.lrange(image_list_id,0,-1)

[b'myBinary6', b'myBinary5', b'myBinary3', b'myBinary3']

In [580]:
redis.flushdb()

True

# Product Catalog Testing

In [41]:
# what a user would put into the product catalog

name = 'ProductA'
description = 'Chicken'
vendor = 'CostCo'
price = 12
currency = 'dollars'
category = 'Meat'
images = 'binaryValue'

In [62]:
import abc, json, pathlib
from abc import abstractmethod
from dataclasses import dataclass, field
import numpy as np
from pathlib import Path
# for type annotations
from typing import Any, Dict, List, Optional, Union, Collection
PathOrStr = Union[Path,str]

In [75]:
# Utils

def if_none(a: Any, b: Any) -> Any:
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a


def to_type(a: Any, data_type):
    """
    If item is None, return None, else, convert to an data_type specified
    (ie. np.array, str, int, float, ect..)
    :parameter: a (Any or None)
    :returns: None or data_type(a)
    """
    return None if a is None else data_type(a)

In [317]:

class DataObject(metaclass=abc.ABCMeta):
    """
    A abstract base class to work with subclasses `DeviationSurvey` and `CalculableObject`.
    """
    @abstractmethod
    def from_json(self):
        pass

    @abstractmethod
    def validate(self):
        pass

    @abstractmethod
    def deserialize(self):
        pass

    @abstractmethod
    def serialize(self):
        pass

In [322]:
@dataclass
class ProductCreate(DataObject):
    """
    Dataclass for Product
    :parameter:
    productId:    (required) product id
    name:         (required) product name
    description:  (required) product description
    vendor:       (required) product vendor
    price:        (required) product price
    currency:     (required) currency
    category:     (required) product category
    images:       (required) images associated with product (binary value)
    :returns:
    dataclassObj: Dataclass Product object
    """

    #productId: str
    name: str
    description: str
    vendor: str
    price: float
    currency: str
    category: str
    productId: str = field(default=None, metadata={'unit': 'str'})
    images: np.ndarray = field(default=None, metadata={'unit': 'str'})

    def from_json(self):
        super().from_json()

    def serialize(self):
        super().serialize()

    def validate(self):
        """
        validate different parameters to ensure that the data in the DataObject
        will work with the directional survey functions
        """


#         def validate_productId(self):
#             """
#             validate that productId is a string
#             :return: pass or TypeError
#             """
#             productId_type = type(self.productId)
#             if productId_type is str:
#                 pass
#             else:
#                 raise TypeError(f"Validation Error: productId has type {productId_type}")
        
        def validate_product_name(self):
            """
            validate that the product name has not been used
            :return: pass or TypeError
            """
            product_name = self.name
            print(product_name)
            if redis.sadd('product-names',product_name) == 1:
                pass
            else:
                raise TypeError(f"Validation Error: product name must be unique: {product_name}")
                
        # run validation functions
#         validate_productId(self)
        validate_product_name(self)

    def deserialize(self):
        """
        convert dict values to their proper deserialized dict values
        converts lists to np.arrays if not None
        converts value to float if not None
        converts value to str if not None
        :parameter:
        DataObject params
        :return:
        DataObject params deserialized as floats, str, int, or np.arrays
        """

        self.productId = to_type(self.productId, str)
        self.name = to_type(self.name, str)
        self.description = to_type(self.description, str)
        self.vendor = to_type(self.vendor, str)
        self.price = to_type(self.price, float)
        self.currency = to_type(self.currency, str)
        self.category = to_type(self.category, str)
        self.images = to_type(self.images, np.array)

    def __post_init__(self):
        """
        validate all data,
        serialized all validated data,
        look in all fields and types,
        if type is None pass,
        else if type given doesnt match dataclass type raise error
        """
        self.validate()
        self.deserialize()
        print(self.deserialize())
        for (name, field_type) in self.__annotations__.items():
            if not isinstance(self.__dict__[name], field_type):
                current_type = type(self.__dict__[name])
                if current_type is type(None):
                    pass
                else:
                    raise ValueError(f"The field `{name}` was assigned by `{current_type}` instead of `{field_type}`")


In [416]:
@dataclass
class ProductUpdate(DataObject):
    """
    Dataclass for Product
    :parameter:
    productId:    (required) product id
    name:         (required) product name
    description:  (required) product description
    vendor:       (required) product vendor
    price:        (required) product price
    currency:     (required) currency
    category:     (required) product category
    images:       (required) images associated with product (binary value)
    :returns:
    dataclassObj: Dataclass Product object
    """

    productId: str
    name: str = field(default=None, metadata={'unit': 'str'})
    description: str = field(default=None, metadata={'unit': 'str'})
    vendor: str = field(default=None, metadata={'unit': 'str'})
    price: float = field(default=None, metadata={'unit': 'float'})
    currency: str = field(default=None, metadata={'unit': 'str'})
    category: str = field(default=None, metadata={'unit': 'str'})
    images: np.ndarray = field(default=None, metadata={'unit': 'str'})

    def from_json(self):
        super().from_json()

    def serialize(self):
        super().serialize()

    def validate(self):
        """
        validate different parameters to ensure that the data in the DataObject
        will work with the directional survey functions
        """


        def validate_productId(self):
            """
            validate that productId is a string
            :return: pass or TypeError
            """
            print(self.productId)
            print(redis.hget(self.productId,'name'))
            if type(redis.hget(self.productId,'name')) is bytes:
                pass
            else:
                raise TypeError(f"Validation Error: productId does not exist")
        
#         def validate_product_name(self):
#             """
#             validate that the product name has not been used
#             :return: pass or TypeError
#             """
#             product_name = self.name
#             print(product_name)
#             if redis.sadd('product-names',product_name) == 1:
#                 pass
#             else:
#                 raise TypeError(f"Validation Error: product name must be unique: {product_name}")
                
        # run validation functions
        validate_productId(self)
        #validate_product_name(self)

    def deserialize(self):
        """
        convert dict values to their proper deserialized dict values
        converts lists to np.arrays if not None
        converts value to float if not None
        converts value to str if not None
        :parameter:
        DataObject params
        :return:
        DataObject params deserialized as floats, str, int, or np.arrays
        """

        self.productId = to_type(self.productId, str)
        self.name = to_type(self.name, str)
        self.description = to_type(self.description, str)
        self.vendor = to_type(self.vendor, str)
        self.price = to_type(self.price, float)
        self.currency = to_type(self.currency, str)
        self.category = to_type(self.category, str)
        self.images = to_type(self.images, np.array)

    def __post_init__(self):
        """
        validate all data,
        serialized all validated data,
        look in all fields and types,
        if type is None pass,
        else if type given doesnt match dataclass type raise error
        """
        self.validate()
        self.deserialize()
        print(self.deserialize())
        for (name, field_type) in self.__annotations__.items():
            if not isinstance(self.__dict__[name], field_type):
                current_type = type(self.__dict__[name])
                if current_type is type(None):
                    pass
                else:
                    raise ValueError(f"The field `{name}` was assigned by `{current_type}` instead of `{field_type}`")


In [406]:
class CalculableObject(DataObject):

    def __init__(self, product_obj, **kwargs):
        """
        DirectionalSurvey object with a wells directional survey info
        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """

        self.product_obj = product_obj

    def validate(self):
        super().validate()

    def deserialize(self):
        super().deserialize()

    @classmethod
    def from_json(cls, path: PathOrStr):
        """
        Pass in a json path, either a string or a Path lib path and convert to a WellboreTrajectory data obj
        :param:
        -------
         path: PathOrStr
        :return:
        -------
        deviation_survey_obj: Obj
        :examples:
        -------
        """

        with open(path) as json_file:
            json_data = json.load(json_file)
        json_file.close()

        res = cls(data=json_data)  # converts json data
        return res

    def serialize(self):
        """
        Convert survey object to serialized json
        :parameter:
        -------
        None
        :return:
        -------
        json: str
        :examples:
        -------
        """
        
        self.productId = to_type(self.productId, str)
        self.name = to_type(self.name, str)
        self.description = to_type(self.description, str)
        self.vendor = to_type(self.vendor, str)
        self.price = to_type(self.price, float)
        self.currency = to_type(self.currency, str)
        self.category = to_type(self.category, str)
        self.images = to_type(self.images, np.array)

        json_obj = dict(productId=str(self.product_obj.productId),
                        name=str(self.product_obj.name),
                        description=str(self.product_obj.description),
                        vendor=str(self.product_obj.vendor),
                        price=float(self.product_obj.price),
                        currency=str(self.product_obj.currency),
                        images=list(self.product_obj.images))


        json_string = json.dumps(json_obj)  # converts a data object into a json string.

        return json_string

In [646]:
class ProductCatalogCreate(CalculableObject):

    def __init__(self, data=None):
        """
        DirectionalSurvey object with a wells directional survey info
        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """

        self.data = data
        self.product_obj = ProductCreate(**self.data)
        
#     def category_mapping(self):
#         "create a mapping for the category"
        
        
    def product_id_gen(self):
        "Generate the product Id"
        product_id_incr = redis.incr('product_id_incr', 1)
        product_id = f"product:{product_id_incr}"
        
        self.product_obj.productId = product_id

    def product_hash(self):
        """
        Calculate TVD, n_s_deviation, e_w_deviation, and dls values along the wellbore
        using md, inc, and azim arrays
        :parameter:
        -------
        None
        :return:
        -------
        calculated np.array values
        tvd: np.array
        dls: np.array
        e_w_deviation: np.array
        n_s_deviation: np.array
        :examples:
        -------
        """
        # get md, inc, and azim arrays
        product_id = self.product_obj.productId
        name = self.product_obj.name
        description = self.product_obj.description
        vendor = self.product_obj.vendor
        price = self.product_obj.price
        currency = self.product_obj.currency
        category = self.product_obj.category
        images = self.product_obj.images
        
        redis.hset(product_id,'name',name)
        redis.hset(product_id,'description',description)
        redis.hset(product_id,'vendor',vendor)
        redis.hset(product_id,'price',price)
        redis.hset(product_id,'currency',currency)
        redis.hset(product_id,'main-category',category)
        
        # create a list of images, keep only 4 per product
        # first image in the list is the number 1 image
        list_id_images = f"{product_id}:images"
        for image_binary_val in images:
            redis.rpush(list_id_images,str(image_binary_val))
            redis.ltrim(list_id_images,0,3)
            #redis.lrange(image_list_id,0,-1)
            
    def category_set(self):
        """
        You can get the ID from the product hash and create the cateogry hash
        you will call the current main cat product id. Find out what it is, then create a unique ID for it.
        Like, MEAT, and then update the set??? idk. ughh
        """
        product_id = self.product_obj.productId
        category = self.product_obj.category
        
        category_id = f"category:{category}"
        redis.sadd(category_id,product_id)
        
#     def image_kv(self):
#         """
#         generate image key value pair
#         """
#         product_id = self.product_obj.productId
#         image = self.product_obj.image
        
        
#         product_image = {f"product_image:{product_id}"

    def generate_product_catalog(self):

        self.product_id_gen()  # get generated product id

        self.product_hash()  # get product hash
        
        self.category_set() # get category set

In [718]:
class ProductCatalogUpdate(CalculableObject):

    def __init__(self, data=None):
        """
        DirectionalSurvey object with a wells directional survey info
        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """

        self.data = data
        self.product_obj = ProductUpdate(**self.data)
        

    def product_hash_update(self):
        """
        Calculate TVD, n_s_deviation, e_w_deviation, and dls values along the wellbore
        using md, inc, and azim arrays
        :parameter:
        -------
        None
        :return:
        -------
        calculated np.array values
        tvd: np.array
        dls: np.array
        e_w_deviation: np.array
        n_s_deviation: np.array
        :examples:
        -------
        """
        # get md, inc, and azim arrays
        product_id = self.product_obj.productId
        name = self.product_obj.name
        description = self.product_obj.description
        vendor = self.product_obj.vendor
        price = self.product_obj.price
        currency = self.product_obj.currency
        category = self.product_obj.category
        images = self.product_obj.images
        
        # update name if not none, this involves removing and replacing a product name set value
        if name is not None: 
            # get the original value of the name in the product id
            orig_name = redis.hget(product_id,'name')
            # remove it from the set product-names, (must decode bytes to string)
            orig_name = orig_name.decode("utf-8")
            redis.srem('product-names', orig_name)
            # add it the new name to the product-names set
            redis.sadd('product-names',name)
            
            # update the name in the product hash
            redis.hset(product_id,'name',name)
            
        # updates if present    
        if description is not None: redis.hset(product_id,'description',description)
        if vendor is not None: redis.hset(product_id,'vendor',vendor)
        if price is not None: redis.hset(product_id,'price',price)
        if currency is not None: redis.hset(product_id,'currency',currency)
        
        # image updates
        if images is not None:
            list_id_images = f"{product_id}:images"
            # reverse the nparray to keep the image order, (1st in list is best image)
            for image_binary_val in images[::-1]:
                # lpush image items in
                redis.lpush(list_id_images,str(image_binary_val))
                # ltrim the list after the 4th image
                redis.ltrim(list_id_images,0,3)
                #redis.lrange(image_list_id,0,-1)
        
    def category_update(self):
        """
        You can get the ID from the product hash and create the cateogry hash
        you will call the current main cat product id. Find out what it is, then create a unique ID for it.
        Like, MEAT, and then update the set??? idk. ughh
        """
        product_id = self.product_obj.productId
        category = self.product_obj.category
        
        if category is not None:
            
            # get the original value of the category
            orig_cat = redis.hget(product_id,'main-category')
            # remove it from the set, (must decode bytes to string)
            orig_cat = orig_cat.decode("utf-8")
            orig_cat_id = f"category:{orig_cat}"
            redis.srem(orig_cat_id, product_id)
            
            # update in the product hash
            redis.hset(product_id,'main-category',category)
            
            # add it to the new set
            category_id = f"category:{category}"
            redis.sadd(category_id,product_id)

            
            
    def product_hash_delete(self):
        """
        Calculate TVD, n_s_deviation, e_w_deviation, and dls values along the wellbore
        using md, inc, and azim arrays
        :parameter:
        -------
        None
        :return:
        -------
        calculated np.array values
        tvd: np.array
        dls: np.array
        e_w_deviation: np.array
        n_s_deviation: np.array
        :examples:
        -------
        """
        # get md, inc, and azim arrays
        product_id = self.product_obj.productId
        

        # get the original value of the name in the product id
        orig_name = redis.hget(product_id,'name')
        # remove it from the set product-names, (must decode bytes to string)
        orig_name = orig_name.decode("utf-8")
        redis.srem('product-names', orig_name)
            
        # delete the hash
        redis.delete(product_id)
        
    def images_list_delete(self):
        
        product_id = self.product_obj.productId
        #images = self.product_obj.images
        

        list_id_images = f"{product_id}:images"
        redis.delete(list_id_images)
                

    def category_remove(self):
        """
        You can get the ID from the product hash and create the cateogry hash
        you will call the current main cat product id. Find out what it is, then create a unique ID for it.
        Like, MEAT, and then update the set??? idk. ughh
        """
        product_id = self.product_obj.productId

        # get the original value of the category
        orig_cat = redis.hget(product_id,'main-category')
        # remove it from the set, (must decode bytes to string)
        orig_cat = orig_cat.decode("utf-8")
        orig_cat_id = f"category:{orig_cat}"
        redis.srem(orig_cat_id, product_id)           

    def update_product_catalog(self):

        self.product_hash_update()  # get product hash
        
        self.category_update() # get category set
        
    def delete_product_catalog(self):
        
        self.category_remove()
        self.product_hash_delete()
        self.images_list_delete()

In [719]:
# testing

In [721]:


# example dict
product_dict = {
    "name": 'Product_Q',
    "description": 'Chicken',
    "vendor": 'WholeFoods',
    "price": 15.5,
    "currency": 'dollars',
    "category": 'Meat',
    "images": [1,2,3,4,5,6,7,8]
}


product_obj = ProductCatalog(product_dict) # get product object
print('product.data:')
print(product_obj.data)
#product_obj.calculate_survey_points() # runs through min curve algo, calc lat lon points, and calc horizontal
#print('calc object:')

product_obj.generate_product_catalog()

Product_Q


TypeError: Validation Error: product name must be unique: Product_Q

In [711]:
redis.hgetall('product:6')

{b'name': b'Product_Q',
 b'description': b'Chicken',
 b'vendor': b'WholeFoods',
 b'price': b'15.5',
 b'currency': b'dollars',
 b'main-category': b'Meat'}

In [712]:
redis.lrange('product:6:images',0,-1)

[b'1', b'2', b'3', b'4']

In [713]:
redis.smembers('category:Meat')

{b'product:1', b'product:2', b'product:4', b'product:6'}

In [714]:
# example dict
product_dict = {
    "productId": 'product:6'
}

product_obj = ProductCatalogUpdate(product_dict) # get product object
product_obj.delete_product_catalog()

product:6
b'Product_Q'
None


In [715]:
redis.hgetall('product:6')

{}

In [716]:
redis.lrange('product:6:images',0,-1)

[]

In [717]:
redis.smembers('category:Meat')

{b'product:1', b'product:2', b'product:4'}

In [532]:
redis.smembers('product-names')

{b'Product_B', b'Product_C', b'product_SICK'}

In [676]:
# example dict
product_dict = {
    "productId": 'product:3',
    "name": 'product_SICK',
    "description": 'ChickenBUTTS',
    "vendor": 'WholeFoodsBUTSS',
    "price": 1202,
    "currency": 'dollarsZZZZ',
    "category": 'Fish',
    "images": [233,2450]
}

product_obj = ProductCatalogUpdate(product_dict) # get product object
product_obj.update_product_catalog()

product:3
b'product_SICK'
None
2450
233


In [677]:
redis.lrange('product:3:images',0,-1)

[b'233', b'2450', b'20', b'100']

In [654]:
redis.hgetall('product:3')

{b'name': b'product_SICK',
 b'description': b'ChickenBUTTS',
 b'vendor': b'WholeFoodsBUTSS',
 b'price': b'1202.0',
 b'currency': b'dollarsZZZZ',
 b'main-category': b'Fish'}

In [655]:
redis.smembers('category:Meat')

{b'product:1', b'product:2'}

In [529]:
redis.smembers('product-names')

{b'Product_B', b'Product_C', b'product_SICK'}

In [477]:
redis.smembers('category:Fish')

{b'product:3', b'product:4', b'product:5', b'product:6', b'product:7'}

In [508]:
redis.flushdb()


True

In [425]:
redis.scan(0)
#redis.smembers('product-names')

(1,
 [b'3748912688',
  b'product:4:images',
  b'product:1:images',
  b'product:2:images',
  b'category:Fish',
  b'product:2',
  b'product:3',
  b'product:5',
  b'product-names',
  b'product:3:images'])

In [384]:
redis.scan(0,'product:1*')

(0, [b'product:1:images', b'product:1'])

In [376]:
redis.scan(5,'product:1*')

(0, [b'product:1'])

In [426]:
redis.hgetall('product:1')

{b'name': b'Product_A',
 b'description': b'Chicken',
 b'vendor': b'WholeFoods',
 b'price': b'15.5',
 b'currency': b'dollars',
 b'main-category': b'Meat'}

In [397]:
redis.smembers('category:Meat')

{b'product:1', b'product:2', b'product:3'}

In [387]:
redis.smembers('product:1:images')

{b'1', b'2', b'3', b'4'}

In [164]:
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0)
r.keys()
    


[]

In [ ]:
# things to do.

"""
Create the cateogry mapping, so you have meat and an Id


"""

In [ ]:
redis()

In [87]:
json_ds = product_obj.serialize() # serialize to json
json_ds

AttributeError: 'ProductCatalog' object has no attribute 'productId'

In [81]:
with redis.pipeline() as pipe:
    for Id, product_dict in product.items():
        print(Id, product_dict)
        for k,v in product_dict.items():
            pipe.hset(Id,k,v)
    pipe.execute()

product:1326692461 {'Name': 'ProductA', 'Description': 'Chicken', 'Vendor': 'CostCo', 'Price': 12, 'Currency': 'dollars', 'MainCategory': 1, 'Images': 1}


In [83]:
print(redis.hgetall("product:1326692461"))

{b'Name': b'ProductA', b'Description': b'Chicken', b'Vendor': b'CostCo', b'Price': b'12', b'Currency': b'dollars', b'MainCategory': b'1', b'Images': b'1'}


In [75]:
redis.get('product-image:1')

In [10]:
for k,v in product.items():
    print(k,v)

Id 1
Name ProductA
Description Chicken
Vendor Walmart
Price 10
Currency dollars
MainCategory 1
Images [1, 2, 3, 4]


In [7]:
# import redis
import redis
import os

# connect redis to localhost and port
redis = redis.Redis(host='localhost', port=6379, db=0)

In [8]:
# simple commands to test
redis.set('foo','bar')
redis.get('foo')

b'bar'

In [48]:
import random

random.seed(444)
# create product dictionary generator.
# identify the product id as "product:(random int)"
# then add each products meta data.
product = {f"product:{random.getrandbits(32)}": i for i in (
    {
    'Name': 'ProductA',
    'Description': 'Chicken',
    'Vendor': 'CostCo',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 1, # mapped to category (Id)
    'Images': 1 # mapped to product_image (Id)
    },
    {
    'Name': 'ProductB',
    'Description': 'Pork',
    'Vendor': 'HEB',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 2, # mapped to category (Id)
    'Images': 1 # mapped to product_image (Id)
    },
    {
    'Name': 'ProductC',
    'Description': 'Duck',
    'Vendor': 'WholeFoods',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 4, # mapped to category (Id)
    'Images': 1 # mapped to product_image (Id)
    },)
}


In [49]:
product

{'product:1326692461': {'Name': 'ProductA', 'Description': 'Chicken', 'Vendor': 'CostCo', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 1, 'Images': 1}, 'product:1236154736': {'Name': 'ProductB', 'Description': 'Pork', 'Vendor': 'HEB', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 2, 'Images': 1}, 'product:56854717': {'Name': 'ProductC', 'Description': 'Duck', 'Vendor': 'WholeFoods', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 4, 'Images': 1}}

In [65]:

with redis.pipeline() as pipe:
    for Id, product_dict in product.items():
        print(Id, product_dict)
        for k,v in product_dict.items():
            pipe.hset(Id,k,v)
    pipe.execute()

product:1326692461 {'Name': 'ProductA', 'Description': 'Chicken', 'Vendor': 'CostCo', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 1, 'Images': 1}
product:1236154736 {'Name': 'ProductB', 'Description': 'Pork', 'Vendor': 'HEB', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 2, 'Images': 1}
product:56854717 {'Name': 'ProductC', 'Description': 'Duck', 'Vendor': 'WholeFoods', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 4, 'Images': 1}


In [66]:
print(redis.hgetall("product:56854717"))

{b'Name': b'ProductC', b'Description': b'Duck', b'Vendor': b'WholeFoods', b'Price': b'10', b'Currency': b'dollars', b'MainCategory': b'4', b'Images': b'1'}


In [64]:
redis.hdel("product:1326692461", 'Id')

1